## Day 7 

In [1]:
using DelimitedFiles

In [63]:
rndict = Dict(1 => 3, 2 => 3, 
              3 => 1, 4 => 1, 
              5 => 2, 6 => 2, 
              7 => 3, 8 => 3,
              9 => 1,
              99 => 0)

struct OpCode
    instr::Int
    mode::Array{Int, 1}
    N::Int
end

function OpCode(c::Int)
    digs = digits(c)
    oc = digs[1] + (length(digs) > 1 ? 10*digs[2] : 0)
    N = rndict[oc]
    mode = zeros(Int, N)
    mode[1:length(digs[3:end])] = digs[3:end]
    return OpCode(oc, mode, N)
end 


function intcode_computer(program::Array{Int,1}, input::Channel{Int}, output::Channel{Int}; pc::Int=1)
    
    Np = length(program)
    Np_new = 10*Np
    for j=1:(Np_new-Np)
        push!(program, 0)
    end
    
    jump_flag = false
    rel_base = 0
    
    function load(addr::Int, mode::Int)::Int
        #println("Load: $(addr) -- $(mode)")
        if mode == 0
            return program[addr+1]
        elseif mode == 1
            return addr
        elseif mode == 2
            return program[addr + rel_base + 1]
        else
            error("Unknown mode $(mode).")
        end
    end
    
    function store(data::Int, addr::Int, mode::Int)
        #println("Store: $(data) --> $(addr) -- $(mode)")
        if mode == 0
            program[addr+1] = data
        elseif mode == 1
            error("Invalid mode 1 for store instruction.")
        elseif mode == 2
            program[addr + rel_base + 1] = data
        else
            error("Unknown mode $(mode).")
        end
    end
    
    function load_input(dst::Int, mode::Int)
        #println("Waiting for next input: store to $(dst) -- $(mode)")
        inp = take!(input)
        store(inp, dst, mode)
    end
    
    function store_output(data::Int)
        #println("Tried to store output $(data)")
        put!(output, data)    
    end    
    
    function jtrue(x::Int, y::Int)
        if x != 0 
            pc = y+1
            jump_flag = true
        end
    end
    
    function jfalse(x::Int, y::Int)
        if x == 0  
            pc = y+1 
            jump_flag = true
        end
    end
    
    lt(x::Int, y::Int)::Int = Int(x < y)
    
    eq(x::Int, y::Int)::Int = Int(x == y)
    
    while program[pc] != 99
        jump_flag = false
        oc = OpCode(program[pc])     
        #println(oc)
        if oc.instr == 1
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            store(x+y, program[pc+3], oc.mode[3])
            
        elseif oc.instr == 2
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            store(x*y, program[pc+3], oc.mode[3])
            
        elseif oc.instr == 3
            load_input(program[pc+1], oc.mode[1])
            
        elseif oc.instr == 4
            x = load(program[pc+1], oc.mode[1])
            store_output(x)
            
        elseif oc.instr == 5
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            jtrue(x, y)
        elseif oc.instr == 6
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            jfalse(x, y)
            
        elseif oc.instr == 7
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            store(lt(x,y), program[pc+3], oc.mode[3])
            
        elseif oc.instr == 8
            x = load(program[pc+1], oc.mode[1])
            y = load(program[pc+2], oc.mode[2])
            store(eq(x,y), program[pc+3], oc.mode[3])
            
        elseif oc.instr == 9
            rel_base += load(program[pc+1], oc.mode[1])
            
        
        else
            @error "Something went wrong! Got code $(oc.instr)."
        end
        
        pc += (jump_flag ? 0 : oc.N+1)
    end
    println("Computer finished!")
    close(input)
    close(output)
end

intcode_computer (generic function with 1 method)

In [64]:
function run_computer(program::Array{Int, 1}, input::Array{Int, 1})
    
    output = Int[]
    input_chan = Channel{Int}(1000)
    output_chan = Channel{Int}(1000)
    
    @sync begin
        @async for inp = input
            put!(input_chan, inp)
        end

        comp_task = @async intcode_computer(copy(program), input_chan, output_chan)

        for outp in output_chan
            push!(output, outp)
        end
    end
    return output
end

run_computer (generic function with 2 methods)

In [65]:
program = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
@time run_computer(program, [8])

Computer finished!
  0.337226 seconds (405.41 k allocations: 19.573 MiB, 1.84% gc time)


1-element Array{Int64,1}:
 1000

In [66]:
program = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
@time output = run_computer(program, [])
program == output

Computer finished!
  0.005601 seconds (1.91 k allocations: 113.864 KiB)


true

In [67]:
program = [1102,34915192,34915192,7,4,7,99,0]
@time output = run_computer(program, [])
length(digits(output[1]))

Computer finished!
  0.000287 seconds (80 allocations: 6.984 KiB)


16

In [68]:
program = [104,1125899906842624,99]
@time output = run_computer(program, [])
output[1] == program[2]

Computer finished!
  0.000277 seconds (68 allocations: 4.891 KiB)


true

In [69]:
input = readdlm("inputs/day9.txt", ',', Int)[:];

In [72]:
@time output = run_computer(input, [1])

Computer finished!
  0.000657 seconds (2.23 k allocations: 350.484 KiB)


1-element Array{Int64,1}:
 3063082071

In [73]:
@time output = run_computer(input, [2])

Computer finished!
  0.330246 seconds (5.12 M allocations: 217.176 MiB, 7.99% gc time)


1-element Array{Int64,1}:
 81348